In [1]:
import pandas as pd
import numpy as np
import os

import wrangle
import anomaly_detection_methods
from env import get_connection

In [2]:
def get_cohort_logs():
    '''
    imports cohort logs and cache's them.
    
    cache's dataframe in a .csv
    '''
    filename = 'cohort_logs.csv'
    
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    
    else:
        query ='''
    SELECT *
    FROM logs
    LEFT JOIN cohorts ON logs.cohort_id = cohorts.id
    ;

                '''
        
        url = get_connection('curriculum_logs')
        df = pd.read_sql(query,url)
        df.to_csv(filename, index=False)
        
        return df

In [3]:
df = get_cohort_logs()

In [4]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0


In [5]:
df.shape

(900223, 15)

In [6]:
cont, cat = wrangle.get_continuous_cat_feats(df)

In [7]:
df.isnull().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id             52893
name           52893
slack          52893
start_date     52893
end_date       52893
created_at     52893
updated_at     52893
deleted_at    900223
program_id     52893
dtype: int64

In [8]:
cont

['user_id', 'cohort_id', 'id']

In [9]:
cat

['deleted_at', 'program_id']

In [10]:
df = df.drop(columns='deleted_at')

In [11]:
df.shape

(900223, 14)

In [12]:
# checking to see if this is the same data and if I can drop one
# they are not the same, there are 104_929 observations when they
## are different
df_test = df.query('created_at != updated_at')

In [13]:
df.id.value_counts()

id
28.0     84031
33.0     40730
29.0     38096
62.0     37109
53.0     36902
24.0     35636
57.0     33844
56.0     33568
51.0     32888
59.0     32015
22.0     30926
58.0     29855
32.0     29356
23.0     28534
52.0     28033
26.0     27749
34.0     26538
25.0     25586
31.0     25359
132.0    23691
55.0     21582
27.0     20743
61.0     17713
134.0    16623
135.0    16397
133.0    14715
14.0      9587
1.0       8890
137.0     8562
21.0      7444
138.0     7276
17.0      4954
13.0      2845
18.0      2158
8.0       1712
139.0     1672
19.0      1237
16.0       755
15.0       691
7.0        598
12.0       302
11.0       253
2.0         93
6.0         72
9.0          5
4.0          4
5.0          1
Name: count, dtype: int64

In [14]:
inner_id,outer_id = anomaly_detection_methods.get_lower_and_upper_bounds(df.id.value_counts())

In [15]:
inner_id

,inner_fence
id,
28.0,84031


In [16]:
outer_id

,outer_fence
id,
